In [ ]:
!pip install -U scikit-learn

In [ ]:
!pip install xgboost

In [ ]:
!pip install sktime --ignore-installed llvmlite

In [ ]:
!pip install pandas_market_calendars

In [ ]:
!pip install -U scikit-optimize

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
feature_df = pd.read_csv('features.csv')

X_train_full = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')
y_train_full = pd.read_csv('train_labels.csv')
y_test = pd.read_csv('test_labels.csv')

print(f'X_train: {X_train_full.shape}, X_test: {X_test.shape}, y_train: {y_train_full.shape}, y_test: {y_test.shape}')

X_train: (3416, 1201), X_test: (855, 1201), y_train: (3416, 2), y_test: (855, 2)


In [3]:
from sktime.forecasting.model_selection import temporal_train_test_split

X_train, X_valid = temporal_train_test_split(X_train_full, test_size=0.12)
y_train, y_valid = temporal_train_test_split(y_train_full, test_size=0.12)

print(f'X_train: {X_train.shape}, X_valid: {X_valid.shape}, y_train: {y_train.shape}, y_valid: {y_valid.shape}')

X_train: (3006, 1201), X_valid: (410, 1201), y_train: (3006, 2), y_valid: (410, 2)


In [4]:
import pandas_market_calendars as mcal

for df in (feature_df, X_train, X_valid, X_test, y_train, y_valid, y_test):
    df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
    df.set_index('date', inplace=True)

nyse = mcal.get_calendar('NYSE')
bday_us = pd.offsets.CustomBusinessDay(holidays=nyse.adhoc_holidays, calendar=nyse.regular_holidays, weekmask="1111100")
feature_df = feature_df.asfreq(bday_us)
X_train = X_train.asfreq(bday_us)
X_valid = X_valid.asfreq(bday_us)
X_test = X_test.asfreq(bday_us)
y_train = y_train.asfreq(bday_us)['ERP_Sign']
y_valid = y_valid.asfreq(bday_us)['ERP_Sign']
y_test = y_test.asfreq(bday_us)['ERP_Sign']

print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}')

X_train: (3006, 1200), X_test: (855, 1200), y_train: (3006,), y_test: (855,)


In [5]:
data_series = pd.concat([y_train, y_valid, y_test])

In [6]:
max_lags = 10
fcst_horizon = [5, 10, 15, 20]
max_horizon = fcst_horizon[-1]
model_vars = feature_df.columns

In [7]:
from skopt.space import Real, Integer
from sklearn import metrics

# Define the parameter grid for cross-validation to search through
param_grid = {
    'min_child_weight': Real(0, 30),
    'max_depth': Integer(3, 30),
    'subsample': Real(0.01, 1.0),
    'colsample_bytree': Real(0.01, 1.0),
    'colsample_bylevel': Real(0.01, 1.0),
    'reg_lambda': Real(1e-5, 100.0, prior='log-uniform'),
    'reg_alpha': Real(1e-5, 10.0, prior='log-uniform'),
    'gamma': Real(1e-5, 1.0, prior='log-uniform'),
    'n_estimators': Integer(50, 500)
}

# Define (negative) log-loss as the loss function, aka scorer
log_loss = metrics.make_scorer(metrics.log_loss, greater_is_better=False, needs_proba=True, labels=[0, 1])

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBClassifier
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
from sktime.utils.plotting import plot_series
import seaborn as sns

# Create empty dataframes and dictionaries to store results
xgb_train_df = pd.DataFrame(
    [(f'h{h}', 'xgb_clf') for h in fcst_horizon], columns=['fh','clf']    
)

xgb_test_df = pd.DataFrame(
    [(f'h{h}', 'xgb_clf') for h in fcst_horizon], columns=['fh','clf']    
)

xgb_pred_df = pd.DataFrame().reindex(data_series.index)
xgb_prob_df = pd.DataFrame().reindex(data_series.index)

xgb_params_dict = {}

xgb_features_dict = {}

# Loop through each forecast horizon (in months) from 1 to 12
i = 0
for h in fcst_horizon:
    p = max_lags + h
    time_start = y_train.index[0] + p*bday_us - 1*bday_us
    mask_Xstart = X_train.index >= time_start
    mask_ystart = y_train.index >= time_start

    # Select only X and y columns and observations for the relevant forecast horizon
    X_train_h = X_train.loc[mask_Xstart, [f'{x}L{lag}' for x in model_vars for lag in range(h, p)]]
    X_valid_h = X_valid[[f'{x}L{lag}' for x in model_vars for lag in range(h, p)]]
    X_test_h = X_test[[f'{x}L{lag}' for x in model_vars for lag in range(h, p)]]
    y_train_h = y_train.loc[mask_ystart]

    # Create an expanding window splitter for cross-validation
    cv = TimeSeriesSplit(n_splits=10)

    # Create an XGB classifier with a fixed learning rate and objective
    # Set the random state to 42 for reproducible results
    xgb_clf = XGBClassifier(learning_rate=0.01, objective='binary:logistic', random_state=42)

    # Create a forecasting random search object with param distributions, scoring and cv as previously defined
    # Fit the search object on the train dataset for the respective forecast horizon
    xgb_search = BayesSearchCV(
        estimator=xgb_clf, search_spaces=param_grid, n_iter=100, verbose=0, scoring=log_loss, cv=cv, random_state=42,
        fit_params = {'early_stopping_rounds': 50, 'eval_metric': 'auc', 'eval_set': (X_valid_h, y_valid)}
    )
    xgb_search.fit(X=X_train_h, y=y_train_h, verbose=0)
    xgb_params_dict[(f'h{h}', 'xgb_clf')] = xgb_search.best_params_
    xgb_fitted = XGBClassifier(**xgb_search.best_estimator_.get_params()).fit(X=X_train_h, y=y_train_h)

    # Calculate the predicted recession binary values and probabilities for the train set
    y_pred_train = xgb_fitted.predict(X_train_h)
    y_prob_train = xgb_fitted.predict_proba(X_train_h)[:, 1]

    xgb_train_df.loc[i, 'Accuracy'] = accuracy_score(y_train_h, y_pred_train)
    xgb_train_df.loc[i, 'F1 Score'] = f1_score(y_train_h, y_pred_train)
    xgb_train_df.loc[i, 'PR AUC'] = average_precision_score(y_train_h, y_prob_train)
    xgb_train_df.loc[i, 'Precision'] = precision_score(y_train_h, y_pred_train)
    xgb_train_df.loc[i, 'Sensitivity'] = recall_score(y_train_h, y_pred_train)
    xgb_train_df.loc[i, 'Specificity'] = recall_score(y_train_h, y_pred_train, pos_label=0)
    xgb_train_df.loc[i, 'ROC AUC'] = roc_auc_score(y_train_h, y_prob_train)

    # Calculate the predicted recession binary values and probabilities for the test set
    y_pred_test = xgb_fitted.predict(X_test_h)
    y_prob_test = xgb_fitted.predict_proba(X_test_h)[:, 1]
    
    xgb_test_df.loc[i, 'Accuracy'] = accuracy_score(y_test, y_pred_test)
    xgb_test_df.loc[i, 'F1 Score'] = f1_score(y_test, y_pred_test)
    xgb_test_df.loc[i, 'PR AUC'] = average_precision_score(y_test, y_prob_test)
    xgb_test_df.loc[i, 'Precision'] = precision_score(y_test, y_pred_test)
    xgb_test_df.loc[i, 'Sensitivity'] = recall_score(y_test, y_pred_test)
    xgb_test_df.loc[i, 'Specificity'] = recall_score(y_test, y_pred_test, pos_label=0)
    xgb_test_df.loc[i, 'ROC AUC'] = roc_auc_score(y_test, y_prob_test)

    # Extract the top 20 features by importance
    feature_importance = pd.Series(xgb_fitted.feature_importances_, index=X_train_h.columns)
    best_features = feature_importance.sort_values(ascending=False).head(20)
    xgb_features_dict[(f'h{h}', 'xgb_clf')] = best_features

    # Convert np arrays to datetime series for time series plotting
    y_pred = pd.concat([pd.Series(y_pred_train).set_axis(y_train_h.index), pd.Series(y_pred_test).set_axis(y_test.index)])
    y_prob = pd.concat([pd.Series(y_prob_train).set_axis(y_train_h.index), pd.Series(y_prob_test).set_axis(y_test.index)])

    xgb_pred_df[(f'h{h}', 'xgb_clf')] = y_pred
    xgb_prob_df[(f'h{h}', 'xgb_clf')] = y_prob

    # Plot the actual recession binary values with the predicted binary values
    plot_series(y_train_h, y_test, y_pred, labels=['y_train', 'y_test', 'y_pred'], x_label='Date', y_label='ERP Sign (Binary)')
    plt.title(f'erp_binary_xgb_h{h}')
    plt.tight_layout()
    plt.savefig(f'erp_binary_xgb_h{h}.png')
    plt.close()

    # Plot the actual recession binary values with the predicted probabilities
    plot_series(y_train_h, y_test, y_prob, labels=['y_train', 'y_test', 'y_prob'], x_label='Date', y_label='Pr(ERP Sign)')
    plt.title(f'erp_prob_xgb_h{h}')
    plt.tight_layout()
    plt.savefig(f'erp_prob_xgb_h{h}.png')
    plt.close()

    # Plot the top 20 features by importance
    # Save all three graphs and close them for matplotlib to release memory
    plt.figure(figsize=(20,28))
    sns.barplot(x=best_features.values, y=best_features.index)
    plt.title(f'feature_importance_xgb_h{h}')
    plt.tight_layout()
    plt.savefig(f'feature_importance_xgb_h{h}.png')
    plt.close()
    
    i += 1

In [ ]:
xgb_train_df.to_csv('erp_xgb_train_results.csv')
xgb_test_df.to_csv('erp_xgb_test_results.csv')
xgb_pred_df.to_csv('erp_xgb_pred_estimates.csv')
xgb_prob_df.to_csv('erp_xgb_prob_estimates.csv')

In [ ]:
xgb_params_df = pd.DataFrame.from_dict(xgb_params_dict, orient='index')
xgb_params_df.to_csv('erp_xgb_parameters.csv')

In [ ]:
# Perform some manipulation and export feature importances to CSV

feature_importance_df = pd.DataFrame()

for h in fcst_horizon:
    tmp_feature = xgb_features_dict[(f'h{h}', 'xgb_clf')].rename_axis(f'h{h}_xgb_feature').reset_index(name=f'h{h}_xgb_importance')
    feature_importance_df = pd.concat([feature_importance_df, tmp_feature.T])

feature_importance_df.to_csv('erp_xgb_feature_importances.csv')